# 03-transformed: Transformation der Talkwalker Daten

* Um was handelt es sich hier  (Kurzbeschreibung Inhalt):  
Die Talkwalker Daten transformieren um die Daten für die Marts vorzubereiten


---
## QUELLEN:  
- Unity-Catalog: 
  - datif_pz_uk_dev.03-transformed.talkwalker_cleaned

## ZIEL  
- Unity-Catalog: 
  - datif_pz_uk_dev.03-transformed.talkwalker_data

  
---
* Versionen (aktuelle immer oben):
-12.08.2025 Minh Hieu Le: Init

In [0]:
%run ../../common/nb_init

In [0]:
# Define schema name and path to store tables
target_schema_name = "03_transformed"
target_path = "talkwalker_transformed"
target_table_name="talkwalker_data"
# Set source and trg path
source_path = sta_endpoint_pz_uk["03_transformed"] + "/talkwalker"

In [0]:
from pyspark.sql.functions import col

In [0]:
spark.sql(
    f"""
    CREATE OR REPLACE VIEW datif_pz_uk_{env}.{target_schema_name}.{target_table_name} AS
    WITH ranked AS (
        SELECT 
            * ,
            ROW_NUMBER() OVER (
                PARTITION BY indexed
                ORDER BY indexed
                --   CASE WHEN subtopic = 'EnBW Vorstand' THEN 1 ELSE 2 END
            ) AS rn
        FROM datif_pz_uk_{env}.`02_cleaned`.talkwalker_cleaned
        WHERE url IS NOT NULL AND url != '' -- AND NOT isnan(url)
          AND lang IN ('de', 'en')
    )
    SELECT 
        TO_TIMESTAMP(published, 'yyyy-MM-dd HH:mm:ss') AS `Veröffentlichungsdatum`,
        TO_TIMESTAMP(indexed, 'yyyy-MM-dd HH:mm:ss') AS `ID`,
        lang AS `Sprache`,
        sentiment AS `Tonalität`,
        INITCAP(regexp_extract(domain_url, 'https?://(?:www\\.)?([^\\.]+)\\.', 1)) AS `Quelle`,
        -- subtopic AS `Themenquelle`,
        source_type[0] AS `QuellTyp`,
        post_type[0] AS `Posttyp`,
        url AS `URL`,
        CASE 
            WHEN source_type[0] != "SOCIALMEDIA" THEN title
            ELSE content
        END AS `Titel`,
        content AS `Kontext`,
        CASE 
            WHEN isnan(engagement) OR engagement IS NULL THEN 0
            ELSE engagement
        END AS Engagement,
        CASE 
            WHEN isnan(reach) OR reach IS NULL THEN 0
            ELSE reach
        END AS `Reichweite`,
        CASE 
            WHEN isnan(estimated_reach) OR estimated_reach IS NULL THEN 0
            ELSE estimated_reach
        END AS `Geschätzte Reichweite`
    FROM ranked
    WHERE rn = 1
    """
)


In [0]:
df = spark.sql(f"SELECT * FROM datif_pz_uk_{env}.{target_schema_name}.{target_table_name}")

display(df)